In [61]:
import random

class Sentiment:
    NEGATIVE  = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
    
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE
        
class ReviewContainer:
    def __init__(self,reviews):
        self.reviews = reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews)) #inorder to filter we just first make it list
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrink = positive[:len(negative)] #just make positive and negative equals
        
        self.reviews = negative + positive_shrink #now store positive_shrink and negative in reviews
        random.shuffle(self.reviews) #now we shuffle negative and positive
#         print(negative[0].text)
#         print(len(negative))
#         print(len(positive))

In [35]:
import json

file_name = './data/sentiment/Books_small_10000.json'

# a = {"hello": "world"}
# print(a['hello'])

reviews = []

with open(file_name) as f:
    for line in f:
        #         print(line) #line is string right now so we have to convert it itno json
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

reviews[5].text
# reviews[5].sentiment
        

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

## Prep Data

In [63]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42) #33% test and 67% training
#total we have 1000 reviews out of which 670 utilize for training and 330 utilize for testing

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

cont.evenly_distribute()

len(cont.reviews)

872

In [78]:
# len(training)
# print(training[0].text)
# len(test) 

train_container.evenly_distribute() #to make both equal we need to call this
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))
# train_x[0]
# train_y[0]
# test_x[0]

436
436


## Bag of Words

In [91]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# vectorizer = CountVectorizer() #now we used TfidfVectorizer instead of count
vectorizer = TfidfVectorizer()


#Same approach
# vactorizer.fit(train_x)
# train_x_vectors = vectorizer.transform(train_x)

train_x_vectors = vectorizer.fit_transform(train_x) # it create bag of vectors of words from train data set
test_x_vectors = vectorizer.transform(test_x) #this is our test data so we just only transform it not fit it
    
# print(train_x[0])
# print(train_x_vectors[0].toarray())

#Now we chose model for inputs:
#train_x_vectors and train_y

#read abour classifiers in sklearn

## Classification

##### Linear SVM

In [92]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

# test_x[36]

clf_svm.predict(test_x_vectors[88])

array(['POSITIVE'], dtype='<U8')

##### Decision Tree

In [93]:
from sklearn.tree import DecisionTreeClassifier

clf_dec =  DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[43])

array(['POSITIVE'], dtype='<U8')

#### Naive Bayes

In [94]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = DecisionTreeClassifier()
clf_gnb.fit(train_x_vectors, train_y)

clf_gnb.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

#### Logisitic Regression

In [95]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)

clf_log.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

### Evaluation

In [96]:
#Mean Accuracy
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors, test_y))
print(clf_log.score(test_x_vectors, test_y))

0.8076923076923077
0.6418269230769231
0.6322115384615384
0.8052884615384616


In [97]:
#F1 Scores
from sklearn.metrics import f1_score
f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]) #very good on positive but ver bad for neg and neutral


array([0.80582524, 0.80952381])

In [101]:
# test_y.count(Sentiment.POSITIVE) #out of 670, 552 are +ve which means very major part
# print(test_y.count(Sentiment.NEGATIVE))

test_set = ['recommended', "bad book to buy", 'horrible waste of time']
new_test = vectorizer.transform(test_set)
clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

### Tuning our Model (with Grid Search)

In [102]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ('linear', 'rbf'), 'C': (1,4,8,16,32)}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(train_x_vectors, train_y)

#cv => cross validate

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [103]:
print(clf.score(test_x_vectors, test_y))

0.8197115384615384


   ### Saving Model

In [107]:
import pickle #we save the model so that we dont train it again

with open('./models/sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)

### Load Model

In [108]:
with open('./models/sentiment_classifier.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)

In [109]:
print(test_x[0])

loaded_clf.predict(test_x_vectors[0])

The writing was ok,but there were so many places where the foul language was so unnecessary that it detracted from the storyline.  The actual concept was great, but in so many ways it just became tedious.


array(['NEGATIVE'], dtype='<U8')